In [ ]:
# Importing (using/include) packages and files needed for the code to run
using SparseArrays
using LinearAlgebra
using IterativeSolvers
using WriteVTK
using Printf
using Statistics
using Dates
using SpecialFunctions
using Roots
using NLsolve
using Printf
using HDF5
include("Grid.jl")
include("Markers.jl")
include("Stokes.jl")
include("GridOperations.jl")
include("Topo.jl")
# Note: that we import pyplot last to avoid a name conflict with grid
using PyPlot
include("Visualization.jl")

# Stefan Problem - test melting and solidification
Step 1 - import packages

Step 2 - Define a function that relates entropy and temperature/pressure. 
$$
T = f_1(P,S)
$$

$$
T = \exp \left( \frac{S}{C_V} + \ln (T_{ref}) - \frac{\Delta H_{fus} }{C_V T_M}X \right)
$$

and 
$$
S = f_2(P,T)
$$
$$
S = C_V (\ln (T) - \ln(T_{ref})) + \frac{\Delta H_{fus}}{T_m}X
$$

In [ ]:
# functions that relates entropy and temperature/pressure are define in TemperatureEntropy.jl file
include("TemperatureEntropy.jl")


Step 3 - Write functions to set up the initial condition for the Stefan problem

- Consider a box with width 100 km and height 100 km.
- The box is filled entirely with one material
- Initially, the top 10 km is solid and the bottom 90 km is liquid.
- The sides are insulating
- Density is constant and equal to 1000 kg/m$^3$
- The surface is at T=100K, the bottom is at T=273 K
- The liquid is isothermal with $T=T_m$.
- The solid has initial $T(z)$ given by the Stefan solution (Turcotte and Schubert section 4.18)
$$
\theta = \frac{erf(\eta)}{erf(\lambda_1)}
$$
- Given the initial temperature, compute initial entropy S on the markers.
  

In [ ]:
function get_interface(grid::CartesianGrid,mat::Matrix{Float64},contour_value::Float64)
    # Finding interfaces
    interface_position = zeros(Float64,grid.nx+1);
    for j in 1:grid.nx+1
        i = 1
        while i <= grid.ny
            if mat[i,j] == contour_value
                interface_position[j] = grid.yc[i]
                break
            elseif mat[i+1,j] < contour_value
                # interface is located within this cell.
                interface_position[j] = grid.yc[i] + (grid.yc[i+1]-grid.yc[i])/(mat[i+1,j]-mat[i,j])*(contour_value-mat[i,j])
                break
            end
            i = i+1
        end
    end
    return interface_position
end

# Main part of code
- At each timestep, begin by computing T(S,X) on the markers
- Iteratively solve the temperature equation. Start with a guess S_new = S_old
    - For T_new, compute $q_{cond} = \nabla \cdot (k \nabla T_{new})$
    - Solve diffusion equation for S_new
    - Given S_new, update T_new
    - Iterate until converged...
- Given the new entropy solution on the cell centers, update the entropy and melt fraction on the markers.

In [ ]:
function model_run(options::Dict)
    W = 1e4
    H = 2e4
    ny = 50
    nx = Int64(ceil(W/H*ny))
    gx = 0.0
    gy = 0.113
    
     # -1 = insulating, 1 = constant temp 
    Tbctype = [-1,-1,1,1] #left, right, top, bottom
    Tbcval = [0.0,0.0,100.0,200.0] #left, right, top, bottom
    markx = 6
    marky = 6
    seconds_in_year = 3.15e7
    plot_interval = 1e3*seconds_in_year # plot interval in seconds
    end_time = 3e7*seconds_in_year
    dtmax = plot_interval
    grid = CartesianGrid(W,H,nx,ny)

    ### Setting up agruments for termination criteria ###
    max_step::Int64=100
    max_time::Float64=-1
    max_time = max_time == -1.0 ? typemax(Float64) : max_time
    max_step = max_step == -1 ? typemax(Int64) : max_step

    time = 0.0
    iout= 0
    last_plot = 0.0
    dt = dtmax
    
    rho_c = nothing
    Hr = nothing
    dTmax = nothing
    dTemp = nothing
    Tnew = nothing
    Tlast = nothing
    Snew = nothing
    Slast = nothing
    Xnew = nothing
    Xlast = nothing
    kThermal_vx = nothing
    kThermal_vy = nothing
    Xlast = nothing
    q_vx = nothing
    q_vy = nothing
   
    kThermal_vx_new = options["thermal conductivity of ice"].*ones(grid.ny+1,grid.nx+1)
    kThermal_vy_new = options["thermal conductivity of ice"].*ones(grid.ny+1,grid.nx+1)
    rho_c_new = options["density of ice"].*ones(grid.ny+1,grid.nx+1)
    Hr_new = zeros(grid.ny+1,grid.nx+1)

    Tlast_new = zeros(grid.ny,grid.nx)
    Slast_new = zeros(grid.ny,grid.nx)
    Xlast_new = zeros(grid.ny,grid.nx)
    for j in 1:grid.nx
        for i in 1:grid.ny
            Tlast_new[i,j] = 200.0 #((2e2-1e2)/2e4)*grid.yc[i]+1e2
            Xlast_new[i,j] = 0.0
            Slast_new[i,j] = compute_S_from_T_X(Xlast_new[i],Tlast_new[i],options)
        end
    end
    
    itime = 1 
    terminate = false
    while !terminate
        # deal with any NaN values from interpolation:
        if itime > 1
            replace_nan!(rho_c,rho_c_new)
            replace_nan!(Hr,Hr_new)
            replace_nan!(Tlast,Tlast_new)
            replace_nan!(Slast,Slast_new)
            replace_nan!(kThermal_vx,kThermal_vx_new)
            replace_nan!(kThermal_vy,kThermal_vy_new)
            replace_nan!(Xlast,Xlast_new)
        end
        # Copy field data
        rho_c = copy(rho_c_new)
        Hr = copy(Hr_new)
        Tlast = copy(Tlast_new)    
        Slast = copy(Slast_new)
        kThermal_vx = copy(kThermal_vx_new)
        kThermal_vy = copy(kThermal_vy_new)
        Xlast = copy(Xlast_new)

        Tlast,Xlast = update_T_X_from_S(Slast,options)
        Tlast,Xlast,Slast = ghost_nodes_center_TXS(grid,Tlast,Xlast,Slast,Tbctype,Tbcval,options)
        
        # Initial guess for S
        Snew = copy(Slast)
        # Initial guess for T
        Tnew,Xnew = update_T_X_from_S(Snew,options)
        Tnew,Xnew,Snew = ghost_nodes_center_TXS(grid,Tnew,Xnew,Snew,Tbctype,Tbcval,options)
        
        last_T_norm = NaN
        T_norm = NaN
        dT = nothing
        dTmax = Inf
        dS = nothing
        dSmax = Inf
        tolerance = 1e-12
        dTnorm = []
        ititer = []
        titer = 1 
        max_titer = 100
        for titer=1:max_titer
            println("Starting iteration $titer, with timestep = ",dt/seconds_in_year," yr, ",dt/seconds_in_year/1e3," Kyr, ",dt/seconds_in_year/1e6," Myr")
            # Computing conductive heat flux
            q_vx,q_vy = compute_q_cond(grid,Tnew,kThermal_vx,kThermal_vy) # W/m^2

            # Computing the new entropy
            Snew = compute_S_new(grid,Tnew,rho_c,Hr,q_vx,q_vy,Slast,dt);
            
            # Updating the new temperature and new melt fraction from the new entropy
            Tnew,Xnew = update_T_X_from_S(Snew,options)
            Tnew,Xnew,Snew = ghost_nodes_center_TXS(grid,Tnew,Xnew,Snew,Tbctype,Tbcval,options)
       
            if titer > 1
                last_T_norm = T_norm;
                T_norm = norm(Tnew[2:grid.ny,2:grid.nx]);
                # println("[",titer,"]: Change in norm is :",T_norm - last_T_norm)
                push!(dTnorm,abs(T_norm-last_T_norm))
                push!(ititer,titer)
            else
                T_norm = norm(Tnew[2:end-1,2:end-1]);
                push!(dTnorm,abs(T_norm-last_T_norm))
                push!(ititer,titer)
            end
            
            # Computing the maximum temperature change
            dT = Tnew - Tlast
            dTmax = maximum(abs.(dT[2:grid.ny,2:grid.nx]))
            # println("Maximum temperature change = $dTmax K")

            # Computing the maximum entropy change
            dS = Snew - Slast
            dSmax = maximum(abs.(dS[2:grid.ny,2:grid.nx]))
            # println("Maximum Entropy change = $dSmax J/K")
            
            # Printing dt
            # println("dt = ",dt/seconds_in_year," yr, ",dt/seconds_in_year/1e3," Kyr, ",dt/seconds_in_year/1e6," Myr")
            # dt = min(dt,dTmax < 10.0 ? dt : dt*10.0/dTmax)

            # println("Tlast at timestep=titer")
            # display(Tlast[1:2,:])
            # display(Tlast)
            
            # println("Tnew at timestep=titer")
            # display(Tnew[1:2,:])
            # display(Tnew)

            # figure()
            # title("At $titer iterations")
            # plot(Tana[:,1],"b-",label="Analytical")
            # plot(Tnew[:,1],"r--",label="Numerical")
            # legend()
            # gca().invert_yaxis()
            # show()

            # Checking for convergence:
            if titer > 1 && abs(T_norm-last_T_norm) < tolerance
                # figure()
                # title("Change in Temperature at timestep=titer")
                # pcolor(grid.xc/1e3,grid.yc/1e3,dT)
                # colorbar(cmap="viridis")
                # gca().invert_yaxis()
                # show()
                
                # figure()
                # title("Change in Entropy at timestep=titer")
                # pcolor(grid.xc/1e3,grid.yc/1e3,dS)
                # colorbar(cmap="viridis")
                # gca().invert_yaxis()
                # show()
                println("Converged after $titer iterations.")                 
                break
            elseif titer == max_titer
                terminate = true
                # error("Did not converged")
                @error("Did not converged")
            elseif any(isnan.(dT))
                terminate = true
                # error("NaN or Inf apperred")
                @error("NaN or Inf apperred")
            end
        end

        Tlast_new = copy(Tnew)
        Xlast_new = copy(Xnew)
        Slast_new = copy(Snew)
        
        figure()
        pcolor(Tnew)
        colorbar(cmap="viridis")
        gca().invert_yaxis()
        show()


        # figure()
        # title("At timestep = $itime")
        # plot(ititer,dTnorm,"-")
        # gca().set_ylabel(L"dTnorm")
        # gca().set_xlabel(L"Iterations")
        # show()
        
        # Checking Termination Criteria, time is in Myr, amplitude is in meters
        if time >= max_time || itime >= max_step 
            terminate = true
        end

        time += dt
        itime += 1
        println("Finished Step ",itime," time = ",time/seconds_in_year," yr, ",time/seconds_in_year/1e3," Kyr, ",time/seconds_in_year/1e6," Myr")  
    end
    return grid
end

In [ ]:
options = Dict()
options["latent heat of fusion"] = 3.34e5 #J/kg
options["specific heat of ice"] = 2.1e3 # J/kg*K (ice)
options["density of ice"] = 1e3 # kg/m^3
options["thermal conductivity of ice"] = 2.2 # W/m*K
options["thermal diffusivity"] = options["thermal conductivity of ice"] / (options["density of ice"]*options["specific heat of ice"]) # 
options["Tm"] = 200.0 # K
# options["To"] = 100.0 # K
# options["ym"] = 1e4 # m 
model_run(options);